In [3]:
import ROOT
import pandas as pd
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

Welcome to JupyROOT 6.22/06


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-qg3tuoun because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# PARAMETERS
---

In [43]:
TAttMarkers = [20,21,22,23,33,34,29,24,25,26,32,27,28,30]
TColors = [1,2,9,8,6]
MarkerSize = 2

# Webplotdigitizer using Harut's MC data
# https://userweb.jlab.org/~avakian/tmp/2022-SIDIS-Aug8.pdf
# A_LL_proton as a function of x
# [x,A_LL_proton]
A_LL_proton = np.array([
[0.11757848013262741, 0.18379497577967974],
[0.1926541526284828, 0.2684978538773837],
[0.2672720466335098, 0.34164975653604424],
[0.3428054976201935, 0.3981517312610664],
[0.4178811701160489, 0.4407159649813103],
[0.4929568426119043, 0.4885217883139879],
[0.5675747366169313, 0.5337541806020067],
[0.6426504091127867, 0.602493032329989]])

# Webplot digitizer using Harut's MC data
# https://userweb.jlab.org/~avakian/tmp/2022-SIDIS-Aug8.pdf
# NH3 dilution factor f as a function of x
# [x,f]
f_NH3 = np.array([
[0.117026744250078, 0.177177027261547],
[0.12562822371923127, 0.17772179947569455],
[0.1342297031883845, 0.17830842392982116],
[0.14290937792543915, 0.1788241341617871],
[0.15111988105508545, 0.17935517777492244],
[0.16003414159584425, 0.17997120009544962],
[0.16863562106499752, 0.18047914233841564],
[0.17739349106995356, 0.18102689485856938],
[0.18583858000330403, 0.1815937981106982],
[0.19444005947245727, 0.18207997718887511],
[0.20304153894161053, 0.18264986074701012],
[0.21195579948236934, 0.18323909763395071],
[0.220244497879917, 0.18375614607129687],
[0.2286895868132675, 0.1843021232034362],
[0.23744745681822355, 0.1848373200515962],
[0.245892545751574, 0.18539417876613007],
[0.25465041575653, 0.18593523492788716],
[0.26325189522568326, 0.1865720820699886],
[0.27154059362323096, 0.18699956671377962],
[0.2804548541639898, 0.1876155890343068],
[0.28905633363314304, 0.1881478055764606],
[0.2976578131022963, 0.1887428004785831],
[0.3058248744164418, 0.1891597826685512],
[0.3151214229336074, 0.1897605829769269],
[0.32330586097395325, 0.1902669950806733],
[0.3319768473479078, 0.19083420400521306],
[0.3406652104480625, 0.19139969796535447],
[0.34895390884561023, 0.19186317553552745],
[0.35778997411846764, 0.19246054828466017],
[0.36670423465922647, 0.19303053314121038],
[0.37514932359257697, 0.1935346580333707],
[0.3844545604728427, 0.19416775713410842],
[0.3927953890489913, 0.19470988729602234],
[0.4014229336074451, 0.19526162007378994],
[0.4099983479872978, 0.1958152425705318],
[0.4188604783494557, 0.19637195733468613],
[0.42798325960461825, 0.1969587506195048],
[0.43598524201986083, 0.19745540786358048],
[0.44562063669793245, 0.19807052442225448],
[0.45404834890508267, 0.19859524428929248],
[0.46335358578534847, 0.199194861598047],
[0.4713381914743907, 0.19967202867159828],
[0.48063473999155637, 0.20033235216572196],
[0.4895490005323152, 0.20092158905266255],
[0.49783769892986285, 0.20136330345804643],
[0.5063609831311147, 0.2019388117439748],
[0.5150406578681693, 0.202477959230329],
[0.5237985278731254, 0.2030684010352613],
[0.5322436168064758, 0.20361772634659897],
[0.5407473521907523, 0.2041412792085024],
[0.5491337946731768, 0.20470968061087763],
[0.5576136370589279, 0.20522004809192537],
[0.5682134400411167, 0.20591869319120468],
[0.57681491951027, 0.20633790868541513],
[0.5854163989794232, 0.2069329035875375],
[0.5941742689843793, 0.20750576745167867],
[0.6026193579177297, 0.20800570711984107],
[0.6112208373868829, 0.20856722022998025],
[0.6198223168560363, 0.20911199244412781],
[0.6284237963251895, 0.209673505554267],
[0.6370252757943428, 0.2101931664244271],
[0.645235778923989, 0.21074346206795286]])
def interp(model,x):
    f = interpolate.interp1d(model[:,0], model[:,1], fill_value='extrapolate')
    a=f(x)
    return a

In [8]:
A_LL_proton_sebastian = pd.read_csv("sebastian_A_LL.txt",sep=" ",names=["xmin","xmax","Q2min","Q2max","A_LL"])

# HELPER FUNCTIONS
---

In [ ]:
# Return list of files for analysis
# Consecutive flag reorganizes file list with increasing run number
def get_files(rootdir = "", target = "",runMin = -1,runMax = -1, runList = [], consecutive=True, monteCarlo=False):
    files = []
    runs = []
    for path in Path(rootdir).glob("{}*/*.root".format(target)):
        runNumber = get_run_from_root(str(path))
        if((runMin < 0 and runMax < 0 and runList==[]) or (runNumber >= runMin and runNumber <= runMax) or (runNumber in runList) or monteCarlo==True):
            tfile = ROOT.TFile(str(path),"READ")
            if(tfile.GetListOfKeys().Contains("tree_postprocess")):
                files.append(str(path))
                runs.append(runNumber)
            else:
                print("ERROR: get_files() found a .root file (run",runNumber,") without a tree_reco...skipping...")
                continue
        else:
            continue
    
    # Sort by runNumber if specified
    if(consecutive==True):
        files = [x for y, x in sorted(zip(runs, files))]
    
    return files

In [3]:
# Grab location of rcdb.csv
def get_rcdb(rootdir = ""):
    rcdb_file = rootdir + "rcdb.csv"
    if(not os.path.exists(rcdb_file)):
        print("ERROR: get_rcdb() cannot find file",rcdb_file,". Aborting...")
        return -1
    return rcdb_file

In [5]:
# Extract the run number from root file name
def get_run_from_root(file):
    runloc = file.index('run')
    start = file.index('-',runloc)
    end = file.index('.',runloc)
    runNumber = int(file[start+1:end])
    return runNumber

# Extract other column value based on runNumber
def get_colval_from_run(rcdb_csv,runNumber,col = ''):
    return rcdb_csv.query('Run=={}'.format(runNumber))[col].values[0]

In [6]:
# Using the rcdb, divide up the file list by polarization
def split_pol_files(files, rcdb_csv):
    positive_files = []
    negative_files = []
    for file in files:
        runNumber = get_run_from_root(file)
        # Grab the value of Tpol for the matching run number
        pol=get_colval_from_run(rcdb_csv,runNumber,'Tpol')
        if(pol>0):
            positive_files.append(file)
        elif(pol<0):
            negative_files.append(file)
        else:
            print("ERROR: Tpol for run",runNumber,"is neither positive or negative. Aborting...")
            return -1
    return positive_files,negative_files

In [7]:
# Using CLAS12Analysis/util/runChargeAsymmetry.csv determine faraday cup accumulation for...
# helicity = +1 events
# helicity = -1 events

def chargeAsymWeights(file,HWP):
    
    runNumber = get_run_from_root(file)
    rCA_df = pd.read_csv('../../../util/runHelicityCounts.csv')
    # Check if this run has had chargeAsymmetry calculated for it
    if(np.sum(rCA_df['Run']==runNumber)==0):
        print("ERROR: Run",runNumber,"does not appear in CLAS12Analysis/util/runHelicityCounts.csv...Aborting...")
        return -1

    # Get accumulated Faraday cup charge for + and - helicity
    # Obtained via HEL::scaler in recon banks
    fcup_pos = get_colval_from_run(rCA_df,runNumber,'fcup_pos')
    fcup_neg = get_colval_from_run(rCA_df,runNumber,'fcup_neg')
 
    npos = get_colval_from_run(rCA_df,runNumber,'npos_recon')
    nneg = get_colval_from_run(rCA_df,runNumber,'nneg_recon')
    
    if(fcup_pos == 0 or fcup_neg == 0):
        print("ERROR: Zero fcup charge (not all runs currently have this)...skipping")
        return False,False
    elif(HWP==1): # HWP out
        return fcup_pos, fcup_neg, npos, nneg
    elif(HWP==0): # HWP in
        return fcup_neg, fcup_pos, nneg, npos


In [8]:
# Creates an RDataFrame Histo1D
def hist(df,direction,name,bins,bintype):
    zhat = 1 # Pointing from target to CLAS12
    histo1d = df.Define("vz","abs(vz_e+4.5)").Filter("helicity=={} && p_e > 2.6 && th_e > 0.14 && th_e < 0.611 && vz < 4".format(zhat*direction)).Histo1D((name,"",len(bins)-1,bins),bintype)
    return histo1d

In [ ]:
# Evaluates average D(y) for each bin
# The TH1 function's GetMeanError is used to extract the error in D(y)
def hist_Dy(df,name,bins,bintype):
    hDy = ROOT.TH1F("hDy","hDy",len(bins)-1,bins) # Binned in desired variable (x, Q2, etc.)
    # Within each kinematic binning, we create a D(y) histogram
    # The number of bins for this histogram is defined below
    nBins_Dy_distribution = 30
    for i in range(len(bins)-1):
        bL = bins[i]
        bR = bins[i+1]
        histo1d = df.Define("vz","abs(vz_e+4.5)").Define("Dy","y*(1-y/2)/(y*y/2+1-y)").Filter("p_e > 2.6 && th_e > 0.14 && th_e < 0.611 && vz < 4 && {} > {} && {} < {}".format(bintype,bL,bintype,bR)).Histo1D((name+"Dy{}".format(i),"",30,0,1),"Dy")
        hdy = histo1d.GetValue().Clone()
        hDy.SetBinContent(i+1,hdy.GetMean())
        hDy.SetBinError(i+1,hdy.GetMeanError())
    return hDy

In [ ]:
def setAxes(tg,axes):
    tg.GetXaxis().SetLimits(axes[0],axes[1])
    tg.GetYaxis().SetRangeUser(axes[2],axes[3])
    tg.GetYaxis().SetLimits(axes[2],axes[3])
    return tg

# UNWEIGHTED AND WEIGHTED A_LL
---

In [54]:
# For a given run, compute the A_LL
def get_A_LL_hist(file,runNumber,df_rcdb,target,bins,bintype,scaleby="FC"):
    # Determine polarization direction of the target
    Tpol = 0
    targetPol = get_colval_from_run(df_rcdb,runNumber,'Tpol')
    HWP = get_colval_from_run(df_rcdb,runNumber,'HWP')
    
    # ************************************************************
    # TEMPORARY PROGRAM AS OF AUGUST 24th 2022
    # Set the targetPol of run b/c RCDB is unreliable at the moment
    # ************************************************************
    
    if(runNumber==16406):
        targetPol = 0.275 # Not correct, just temp

    if(targetPol>0):# Extract 
        Tpol = 1
    else:
        Tpol = -1
    
    if(HWP==0): #If the HWP is in, artificially flip Tpol sign
        Tpol = Tpol * -1
    
    # Get accumulated Faraday cup charge for + and - helicity
    fcup_pos, fcup_neg,npos_recon,nneg_recon = chargeAsymWeights(file,HWP)
    if(fcup_pos==False):
        return -1
    
    
        
    # Create RDataframe from file
    ROOT.EnableImplicitMT()
    df = ROOT.RDataFrame("tree_postprocess",file)
    
    # Create histograms for anti-parallel/parallel events
    h_ap = hist(df,-Tpol,"h_{}_antiparallel".format(runNumber),bins,bintype)
    h_p = hist(df,Tpol,"h_{}_parallel".format(runNumber),bins,bintype)
    
    # Create D(y) histogram
    h_Dy = hist_Dy(df,"h_{}.format(runNumber)",bins,bintype)
    
    # Process histograms (from Histo1D --> TH1D)
    H_ap = h_ap.GetValue().Clone()
    H_p = h_p.GetValue().Clone()
    Hw_ap = h_ap.GetValue().Clone() # To be weighted by beam Charge Asym
    Hw_p = h_p.GetValue().Clone()   # To be weighted by beam Charge Asym
    H_Dy = h_Dy.Clone()
    
    H_ap.Sumw2()
    H_p.Sumw2()
    Hw_ap.Sumw2()
    Hw_p.Sumw2()
    H_Dy.Sumw2()
    
    # Weigh histograms by helicity counts
    # MIGHT'VE MADE A MISTAKE HERE
    # Edited 9/11/2022
    
    if(Tpol==1):
        Hw_ap.Scale(1/fcup_neg)
        Hw_p.Scale(1/fcup_pos)
    else:
        Hw_ap.Scale(1/fcup_pos)
        Hw_p.Scale(1/fcup_neg)
    
    
    # Grab N+ and N- along with their errors
    N_p = [H_p.GetBinContent(i+1) for i in range(H_p.GetNbinsX())]
    N_m = [H_ap.GetBinContent(i+1) for i in range(H_p.GetNbinsX())]
    Nw_p = [Hw_p.GetBinContent(i+1) for i in range(H_p.GetNbinsX())]
    Nw_m = [Hw_ap.GetBinContent(i+1) for i in range(H_p.GetNbinsX())]
    
    N_err_p = [H_p.GetBinError(i+1) for i in range(H_p.GetNbinsX())]
    N_err_m = [H_ap.GetBinError(i+1) for i in range(H_p.GetNbinsX())]
    Nw_err_p = [Hw_p.GetBinError(i+1) for i in range(H_p.GetNbinsX())]
    Nw_err_m = [Hw_ap.GetBinError(i+1) for i in range(H_p.GetNbinsX())]
    
    # Scale by float to avoid division issues
    H_ap.Scale(1.0)
    H_p.Scale(1.0)
    
    # Create numerator and denominator histogram
    H_numerator = H_p.Clone()
    H_denominator = H_p.Clone()
    Hw_numerator = Hw_p.Clone()
    Hw_denominator = Hw_p.Clone()
    
    # Construct numerator and denominator of asymmetry
    H_numerator.Add(H_ap,-1)
    H_denominator.Add(H_ap,1)
    Hw_numerator.Add(Hw_ap,-1)
    Hw_denominator.Add(Hw_ap,1)
    
    # Divide numerator and denominator
    H = H_numerator.Clone()
    H.Divide(H_denominator)
    Hw = Hw_numerator.Clone()
    Hw.Divide(Hw_denominator)
    
    # Label Histogram
    H.SetTitle("{} asymmetries RG-C;{};".format(target,bintype)+"(N^{+}-N^{-})/(N^{+}+N^{-})")
    Hw.SetTitle("{} asymmetries RG-C;{};".format(target,bintype)+"(L^{-}N^{+}-L^{+}N^{-})/(L^{-}N^{+}+L^{+}N^{-})")
    
    # Return run and H
    return [runNumber,target,targetPol,HWP,H.Clone(),Hw.Clone(),fcup_pos,fcup_neg,npos_recon,nneg_recon,N_p,N_m,N_err_p,N_err_m,Nw_p,Nw_m,Nw_err_p,Nw_err_m,H_Dy]

# MAIN CODE
---

In [10]:
def compute_asym(rootdir = "",
                 target  = "",
                 bins = np.array([]),
                 bintype = "",
                 runMin = -1,
                 runMax = -1,
                 runList = [],
                 doRunOrdering = True,
                 scaleby="FC"):
    # Check if user both defined a runList and runRange
    if(runMin>0 and runMax>0 and runList):
        print("ERROR: compute_asym must have EITHER a run range OR a run list defined, not both...Aborting...")
        return -1
    
    # Obtain path to all .root files for analysis
    files = get_files(rootdir, target, runMin, runMax, runList, consecutive = doRunOrdering)
    if(files==[]):
        print("ERROR: compute_asym could not find any files. Aborting...")
        return -1
    
    # List of runs
    runs = []
    for file in files:
        runs.append(get_run_from_root(file))
    
    # Obtain rcdb file and load as pandas csv
    rcdb = get_rcdb(rootdir)
    df_rcdb = pd.read_csv(rcdb)
    

    # Construct A_LL histograms run-by-run
    ret = []
    for run,file in zip(runs,files):
        ret.append(get_A_LL_hist(file,run,df_rcdb,target,bins,bintype,scaleby))
        print("Completed run",run)
    print("Done")
    return ret

# PLOTTING TOOLS (ASYMMETRIES)
---

In [ ]:
# Assign unique markers and colors to array of TGraphErrors
def stylize_tgrapherrors(tge):
    nMarkers = len(TAttMarkers)
    nColors = len(TColors)
    nPlots = len(tge)
    if(nPlots > nColors * nMarkers):
        print("ERROR: The number of plots sent to stylize_tgrapherrors is too large (",nPlots,">",nColors*nMarkers,")...Aborting...")
        return -1
    
    for i in range(nColors):
        color = TColors[i]
        for j in range(nMarkers):
            marker = TAttMarkers[j]
            if(nMarkers*i+j>=nPlots):
                return tge
            else:
                tge[nMarkers*i+j].SetMarkerStyle(marker)
                tge[nMarkers*i+j].SetMarkerColor(color)
                tge[nMarkers*i+j].SetMarkerColor(color)
                tge[nMarkers*i+j].SetMarkerSize(MarkerSize)  
    return tge
    

In [ ]:
# From an array of plots, get the minimum and maximum y values
# The 'error' parameter is for error bar plots
def get_plot_minmax(plotArr,error, expand = 0.1):
    plotmin, plotmax = 999,-999
    for plot in plotArr:
        for point_idx in range(plot.GetN()):
            tempUp, tempDown = 0,0
            if(error==True):
                tempUp = plot.GetPointY(point_idx) + plot.GetErrorY(point_idx)
                tempDown = plot.GetPointY(point_idx) - plot.GetErrorY(point_idx)
            else:
                tempUp = plot.GetPointY(point_idx)
                tempDown = tempUp
            
            if(tempUp>plotmax):
                plotmax = tempUp
            if(tempDown<plotmin):
                plotmin = tempDown
                
    # Expand range by expand%
    plotRange = plotmax-plotmin
    exp = plotRange * expand
    plotmin = plotmin - 0.5 * exp
    plotmax = plotmax + 0.5 * exp
    
    # Return
    return plotmin,plotmax            

# TOOLS (TARGET POLARIZATION)
---

In [10]:
def unpack_to_dataframe(H):
    runNumberArr = [a[0] for a in H]
    TargArr = [a[1] for a in H]
    TpolArr = [a[2] for a in H]
    HWPArr = [a[3] for a in H]
    histArr = [a[4] for a in H]
    histwArr = [a[5] for a in H]
    fcup_posArr = [a[6] for a in H]
    fcup_negArr = [a[7] for a in H]
    nposArr = [a[8] for a in H]
    nnegArr = [a[9] for a in H]
    NpArr = [a[10] for a in H]
    NmArr = [a[11] for a in H]
    NperrArr = [a[12] for a in H]
    NmerrArr = [a[13] for a in H]
    NwpArr = [a[14] for a in H]
    NwmArr = [a[15] for a in H]
    NwperrArr = [a[16] for a in H]
    NwmerrArr = [a[17] for a in H]
    histDyArr = [a[18] for a in H]
    
    # Create column names for pandas dataframe
    cols = ["Run","Target","Tpol","HWP","binType","min","max"]
    xtitle = histArr[0].GetXaxis().GetTitle()
    cols.extend(["A_LL","A_LL_err","A_LL_wt","A_LL_wt_err","A_LL_theory","D","D_err","f","fcup_pos","fcup_neg","npos_recon","nneg_recon","N+","N-","N+err","N-err","n+","n-","n+err","n-err"])
    
    # Create Pandas Dataframe
    df = pd.DataFrame(columns=cols)
    
    # Append rows to dataframe
    for r,targ,tpol,hwp,h,hw,Np,Nn,npos,nneg,_np,_nm,_nwp,_nwm,_nperr,_nmerr,_nwperr,_nwmerr,_dy in zip(runNumberArr, TargArr, TpolArr, HWPArr,histArr,histwArr,
                                                                                                        fcup_posArr,fcup_negArr,nposArr,nnegArr,
                                                                                                        NpArr,NmArr,NwpArr,NwmArr,
                                                                                                        NperrArr,NmerrArr,NwperrArr,NwmerrArr,histDyArr):
        row = []
        for bin_idx in range(h.GetNbinsX()):
            bc = h.GetBinCenter(bin_idx+1)
            bw = h.GetBinWidth(bin_idx+1)
            bl = bc-0.5*bw
            br = bc+0.5*bw
            
            a_ll = h.GetBinContent(bin_idx+1)
            a_ll_err = h.GetBinError(bin_idx+1)
            
            a_ll_wt = hw.GetBinContent(bin_idx+1)
            a_ll_wt_err = hw.GetBinError(bin_idx+1)
            
            a_ll_theory = interp(A_LL_proton,bc)
            
            D = _dy.GetBinContent(bin_idx+1)
            Derr = _dy.GetBinError(bin_idx+1)
            
            f = np.round(interp(f_NH3,bc),6)
            
            if(hwp==0):
                hwp="in"
            else:
                hwp="out"
            row = [r,targ,tpol,hwp,xtitle,bl,br,a_ll,a_ll_err,a_ll_wt,a_ll_wt_err,a_ll_theory,
                   D,Derr,f,Np,Nn,npos,nneg,
                   _np[bin_idx],_nm[bin_idx],_nperr[bin_idx],_nmerr[bin_idx],
                   _nwp[bin_idx],_nwm[bin_idx],_nwperr[bin_idx],_nwmerr[bin_idx]]
            
            # Add row
            df.loc[len(df.index)] = row
    
    # Return dataframe
    return df

In [ ]:
# Calculates the Pt using Sebastian's bin-by-bin weighing scheme
def calc_Pt(xminArr,xmaxArr,nruns,reps,Np,Np_err,Nm,Nm_err,A_th,beamPol,D,D_err,f):

    xcenterArr = (xmaxArr+xminArr)/2
    
    Pt_Arr = []
    Pt_err_Arr = []
    
    for i in range(nruns):
        
        _s = slice(i*reps,(i+1)*reps)
        
        numerator = np.sum((Np[_s]-Nm[_s])*A_th[_s]*D[_s]*f[_s])
        denominator = np.sum((Np[_s]+Nm[_s])*A_th[_s]*A_th[_s]*D[_s]*D[_s]*f[_s]*f[_s])

        numerator_err = np.sqrt(np.sum(  (A_th[_s]*f[_s])**2 * (D[_s]**2*Np_err[_s]**2+D[_s]**2*Nm_err[_s]**2+(Np[_s]-Nm[_s])**2*D_err[_s]**2)     ))
        denominator_err = np.sqrt(np.sum(  (A_th[_s]*f[_s])**4 * (D[_s]**4*Np_err[_s]**2+D[_s]**4*Nm_err[_s]**2+4*(Np[_s]+Nm[_s])**2*D[_s]**2*D_err[_s]**2)     ))
        
        Pt_Arr.append(numerator/denominator/beamPol)
        Pt_err_Arr.append(np.sqrt(numerator**2*denominator_err**2+denominator**2*numerator_err**2)/beamPol/denominator/denominator)

    return Pt_Arr, Pt_err_Arr

In [1]:
font = {'size'   : 15}
plt.rc('font', **font)

markerColors=['k','r','b','g','y']
markerStyles=['.','o','v','^','<','>','s','P','*','X','x','d']
markers = [mc+ms for mc in markerColors for ms in markerStyles]

plt.rcParams['text.usetex'] = True
def multigraph(df,beamPol,scaleby="FC"):
    
    # Extract arrays
    runNumberArr = df["Run"].to_numpy()
    TargArr = df["Target"].to_list()
    TpolArr = df["Tpol"].to_numpy()
    HWPArr = df["HWP"].to_list()
    binTypeArr = df["binType"].to_list()
    xminArr = df["min"].to_numpy()
    xmaxArr = df["max"].to_numpy()
    A_Arr = df["A_LL"].to_numpy()
    A_err_Arr = df["A_LL_err"].to_numpy()
    A_wt_Arr = df["A_LL_wt"].to_numpy()
    A_wt_err_Arr = df["A_LL_wt_err"].to_numpy()
    A_th_Arr = df["A_LL_theory"].to_numpy()
    D_Arr = df["D"].to_numpy()
    D_err_Arr = df["D_err"].to_numpy()
    dilution_Arr = df["f"].to_numpy()
    FCup_posArr = df["fcup_pos"].to_numpy() 
    FCup_negArr = df["fcup_neg"].to_numpy() 
    npos_reconArr = df["npos_recon"].to_numpy()
    nneg_reconArr = df["nneg_recon"].to_numpy()
    Np_Arr = df["N+"].to_numpy()
    Nm_Arr = df["N-"].to_numpy()
    Nwp_Arr = df["n+"].to_numpy()
    Nwm_Arr = df["n-"].to_numpy()
    Np_err_Arr = df["N+err"].to_numpy()
    Nm_err_Arr = df["N-err"].to_numpy()
    Nwp_err_Arr = df["n+err"].to_numpy()
    Nwm_err_Arr = df["n-err"].to_numpy()
    
    # Extract unique arrays (b/c of repetitive vals)
    reps = np.sum(TpolArr==TpolArr[0]) # Number of kinematic bins (e.g. xbins)
    urunNumberArr = runNumberArr[::reps]
    uTargArr = TargArr[::reps]
    uTpolArr = TpolArr[::reps]
    uHWPArr = HWPArr[::reps]
    uFCup_posArr = FCup_posArr[::reps]
    uFCup_negArr = FCup_negArr[::reps]
    unpos_reconArr = npos_reconArr[::reps]
    unneg_reconArr = nneg_reconArr[::reps]
    nruns = len(uTargArr)

    
    # Bools for selection
    bool_negPol = (uTpolArr<0)
    bool_HWPin  = np.array([(hwp=="in") for hwp in uHWPArr])
    bool_HWPout = np.array([(hwp=="out") for hwp in uHWPArr])
    
    
    fig1, axs1 = plt.subplots(1,2,figsize=(20, 10))
    # Tpol vs. run
    axs1[0].plot(urunNumberArr[bool_HWPin],uTpolArr[bool_HWPin],"k.",label="HWP in")
    axs1[0].plot(urunNumberArr[(bool_negPol * bool_HWPin)],-uTpolArr[(bool_negPol * bool_HWPin)],"r.")
    axs1[0].plot(urunNumberArr[bool_HWPout],uTpolArr[bool_HWPout],"kx",label="HWP out")
    axs1[0].plot(urunNumberArr[(bool_negPol * bool_HWPout)],-uTpolArr[(bool_negPol * bool_HWPout)],"rx")
    axs1[0].set_xlabel("Run")
    axs1[0].set_ylabel("Tpol")
    axs1[0].grid()
    axs1[0].legend()
    
    if(scaleby=="FC"):
        # Faraday cup ratio vs. run
        uFCup_ratio = (uFCup_posArr-uFCup_negArr)/(uFCup_posArr+uFCup_negArr)
        axs1[1].plot(urunNumberArr[bool_HWPin],uFCup_ratio[bool_HWPin],"k.",label="HWP in")
        axs1[1].plot(urunNumberArr[bool_HWPout],uFCup_ratio[bool_HWPout],"kx",label="HWP out")
        axs1[1].set_xlabel("Run")
        axs1[1].set_ylabel("(FC+  -  FC-)/(FC+  +  FC-)")
        axs1[1].grid()
        axs1[1].legend()
    elif(scaleby=="RECON"):
        # Helicity count ratio vs. run
        uHel_ratio = (unpos_reconArr-unneg_reconArr)/(unpos_reconArr+unneg_reconArr)
        axs1[1].plot(urunNumberArr[bool_HWPin],uHel_ratio[bool_HWPin],"k.",label="HWP in")
        axs1[1].plot(urunNumberArr[bool_HWPout],uHel_ratio[bool_HWPout],"kx",label="HWP out")
        axs1[1].set_xlabel("Run")
        axs1[1].set_ylabel(r"($N_{r}$+  -  $N_{r}$-)/($N_{r}$+  +  $N_{r}$-)")
        axs1[1].grid()
        axs1[1].legend()
    
    # Unweighted A_LL
    fig2,axs2 = plt.subplots(1,1,figsize=(20,10))
    Nruns = len(urunNumberArr)
    for i,r,xmin,xmax,a_ll,a_ll_err in zip(range(len(runNumberArr)),runNumberArr,xminArr,xmaxArr,A_Arr,A_err_Arr):
        idx = np.floor(i/reps)
        xrange = xmax-xmin
        xcenter = (xmax+xmin)/2
        xl = xcenter-xrange/4
        xr = xcenter+xrange/4
        x = xl+idx*(xr-xl)/Nruns
        y = a_ll
        yerr = a_ll_err
        
        if(i%reps==0):
            axs2.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)],label=str(r))
        else:
            axs2.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)])
    axs2.grid()
    axs2.set_xlabel(binTypeArr[0],fontsize=20)
    axs2.set_ylabel(r"Unweighted $A_{LL} = \frac{N^{+}-N^{-}}{N^{+}+N^{-}}$",fontsize=30)
    axs2.legend(ncol=4)
    
    # Weighted A_LL
    fig3,axs3 = plt.subplots(1,1,figsize=(20,10))
    for i,r,xmin,xmax,a_ll,a_ll_err in zip(range(len(runNumberArr)),runNumberArr,xminArr,xmaxArr,A_wt_Arr,A_wt_err_Arr):
        idx = np.floor(i/reps)
        xrange = xmax-xmin
        xcenter = (xmax+xmin)/2
        xl = xcenter-xrange/4
        xr = xcenter+xrange/4
        x = xl+idx*(xr-xl)/Nruns
        y = a_ll
        yerr = a_ll_err
        
        if(i%reps==0):
            axs3.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)],label=str(r))
        else:
            axs3.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)])
    axs3.grid()
    axs3.set_xlabel(binTypeArr[0],fontsize=20)
    axs3.set_ylabel(r"Weighted $A_{LL} = \frac{L^{-}N^{+}-L^{+}N^{-}}{L^{-}N^{+}+L^{+}N^{-}}$",fontsize=30)
    axs3.legend(ncol=4)
    
    # Theory A_LL
    fig4,axs4 = plt.subplots(1,1,figsize=(20,10))
    x = np.linspace(0.05,0.7,100)
    y = np.array([interp(A_LL_proton,X) for X in x])
    axs4.plot(x,y,"k-",label="Extrapolation")
    axs4.plot(A_LL_proton[:,0],A_LL_proton[:,1],"ro",label="Harut data",markersize=10)
    axs4.grid()
    axs4.legend()
    axs4.set_xlabel(binTypeArr[0],fontsize=20)
    axs4.set_ylabel(r"Theory $A_{LL}$",fontsize=30)
    
    # Depolarization
    fig5,axs5 = plt.subplots(1,1,figsize=(20,10))
    Nruns = len(urunNumberArr)
    for i,r,xmin,xmax,dy,dye in zip(range(len(runNumberArr)),runNumberArr,xminArr,xmaxArr,D_Arr,D_err_Arr):
        idx = np.floor(i/reps)
        xrange = xmax-xmin
        xcenter = (xmax+xmin)/2
        xl = xcenter-xrange/4
        xr = xcenter+xrange/4
        x = xl+idx*(xr-xl)/Nruns
        y = dy
        yerr = dye
        
        if(i%reps==0):
            axs5.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)],label=str(r))
        else:
            axs5.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)])
    axs5.grid()
    axs5.set_xlabel(binTypeArr[0],fontsize=20)
    axs5.set_ylabel(r"Average D(y)",fontsize=30)
    axs5.legend(ncol=4)
    
    
    # Dilution
    fig6,axs6 = plt.subplots(1,1,figsize=(20,10))
    Nruns = len(urunNumberArr)
    for i,r,xmin,xmax,f in zip(range(len(runNumberArr)),runNumberArr,xminArr,xmaxArr,dilution_Arr):
        idx = np.floor(i/reps)
        xrange = xmax-xmin
        xcenter = (xmax+xmin)/2
        x = xcenter
        y = f
        yerr = 0
        
        if(i%nruns==0):
            axs6.errorbar(x,y,yerr=yerr,fmt="ko-")
        else:
            continue
    axs6.grid()
    axs6.set_xlabel(binTypeArr[0],fontsize=20)
    axs6.set_ylabel(r"Dilution factor",fontsize=30)
    
    # Pt version 1 (bin-by-bin)
    fig7,axs7 = plt.subplots(1,1,figsize=(20,10))
    for i,r,xmin,xmax,a_ll,a_ll_err,a_ll_th,dy,f in zip(range(len(runNumberArr)),runNumberArr,xminArr,xmaxArr,A_wt_Arr,A_wt_err_Arr,A_th_Arr,D_Arr,dilution_Arr):
        idx = np.floor(i/reps)
        xrange = xmax-xmin
        xcenter = (xmax+xmin)/2
        xl = xcenter-xrange/4
        xr = xcenter+xrange/4
        x = xl+idx*(xr-xl)/Nruns
        y = a_ll/(beamPol*dy*f*a_ll_th)
        yerr =a_ll_err/(beamPol*dy*f*a_ll_th)
        
        if(i%reps==0):
            axs7.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)],label=str(r))
        else:
            axs7.errorbar(x,y,yerr=yerr,fmt=markers[int(idx)])
    axs7.grid()
    axs7.legend(ncol=4)
    axs7.set_xlabel(binTypeArr[0],fontsize=20)
    axs7.set_ylabel(r"$P_{t}=A_{LL}^{RAW}/(P_b*D(y)*f*A_{LL}^{TH})$",fontsize=30)
    axs7.text(0.4,0.25,r"$P_b={:.1f}\%$".format(100*beamPol),fontsize=25)
    
    # Pt version 2 (Sebastian's scheme)
    fig8,axs8 = plt.subplots(1,1,figsize=(20,10))
    
    Pt_w_Arr, Pt_w_err_Arr = calc_Pt(xminArr,xmaxArr,nruns,reps,Nwp_Arr,Nwp_err_Arr,Nwm_Arr,Nwm_err_Arr,A_th_Arr, 0.83, D_Arr, D_err_Arr,dilution_Arr)
    Pt_Arr, Pt_err_Arr = calc_Pt(xminArr,xmaxArr,nruns,reps,Np_Arr,Np_err_Arr,Nm_Arr,Nm_err_Arr,A_th_Arr, 0.83, D_Arr, D_err_Arr,dilution_Arr)
    
    axs8.errorbar(urunNumberArr,Pt_w_Arr,yerr=Pt_w_err_Arr,label="FCup weights",fmt="ro-")
    axs8.errorbar(urunNumberArr,Pt_Arr,yerr=Pt_err_Arr,label="No FCup weights",fmt="ko-")
    
    axs8.grid()
    axs8.legend()
    axs8.set_xlabel(binTypeArr[0],fontsize=20)
    axs8.set_ylabel(r"$P_{t}=\frac{1}{P_b}\frac{\sum_x\left[(N^{+}-N^{-}) A_{th}(x)\bar{D(y)}f(x)\right]}{\sum_x\left[(N^{+}+N^{-}) A^{2}_{th}(x)\bar{D(y)}^{2}f^{2}(x)\right]}$",fontsize=30)
    axs8.text(0.55,0.55,r"$P_b={:.1f}\%$".format(100*beamPol),fontsize=25,horizontalalignment='center',
     verticalalignment='center',transform=axs8.transAxes)
    
    # Pt version 2 (Sebastian's scheme) but SMALL
    fig9,axs9 = plt.subplots(1,1,figsize=(10,10))
    
    Pt_Arr, Pt_err_Arr = calc_Pt(xminArr,xmaxArr,nruns,reps,Np_Arr,Np_err_Arr,Nm_Arr,Nm_err_Arr,A_th_Arr, 0.83, D_Arr, D_err_Arr,dilution_Arr)
    
    axs9.errorbar(urunNumberArr,Pt_Arr,yerr=Pt_err_Arr,label="No FCup weights",fmt="ko-")
    
    axs9.grid()
    axs9.legend()
    axs9.set_xlabel(binTypeArr[0],fontsize=20)
    axs9.set_ylabel(r"$P_{t}=\frac{1}{P_b}\frac{\sum_x\left[(N^{+}-N^{-}) A_{th}(x)\bar{D(y)}f(x)\right]}{\sum_x\left[(N^{+}+N^{-}) A^{2}_{th}(x)\bar{D(y)}^{2}f^{2}(x)\right]}$",fontsize=30)
    axs9.text(0.55,0.8,r"$P_b={:.1f}\%$".format(100*beamPol),fontsize=25,horizontalalignment='center',
     verticalalignment='top',transform=axs9.transAxes)
    
    
    
    # Return stuff
    DF = df
    DF = DF.drop(['npos_recon'],axis=1)
    DF = DF.drop(['nneg_recon'],axis=1)
    DF["Pb"]=np.ones(reps*nruns)*beamPol
    DF["Pt"]=np.repeat(Pt_Arr,reps)
    DF["Pt_err"]=np.repeat(Pt_err_Arr,reps)
    DF.rename(columns={'Tpol':'Tpol RCDB'}, inplace=True)
    DF = DF.round({'f' : 4})
    return DF

NameError: name 'plt' is not defined